In [141]:
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA

In [142]:
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [143]:
def plot_hyperplane(clf, min_x, max_x, linestyle, label):
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(min_x - 5, max_x + 5)
    yy = a * xx - (clf.intercept_[0]) / w[1]
    plt.plot(xx, yy, linestyle, label=label)

In [144]:
def plot_subfigure(X, Y, subplot, title, transform):
    if transform == "pca":
        X = PCA(n_components=2).fit_transform(X)
    elif transform == "cca":
        X = CCA(n_components=2).fit(X, Y).transform(X)
    else:
        raise ValueError

    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    classif = OneVsRestClassifier(SVC(kernel='linear'))
    classif.fit(X, Y)
 
    print title,"准确率:", classif.score(X, Y)

    ## 画图
    plt.subplot(2, 2, subplot)
    plt.title(title)

    zero_class = np.where(Y[:, 0])
    one_class = np.where(Y[:, 1])
    plt.scatter(X[:, 0], X[:, 1], s=40, c='gray')
    plt.scatter(X[zero_class, 0], X[zero_class, 1], s=160, edgecolors='b',
               facecolors='none', linewidths=2, label=u'类别1')
    plt.scatter(X[one_class, 0], X[one_class, 1], s=80, edgecolors='orange',
               facecolors='none', linewidths=2, label=u'类别2')
    plot_hyperplane(classif.estimators_[0], min_x, max_x, 'k--', u'类别1边界')
    plot_hyperplane(classif.estimators_[1], min_x, max_x, 'r-.', u'类别2边界')
    
    plt.xticks(())
    plt.yticks(())
    plt.xlim(min_x - .5 * max_x, max_x + .5 * max_x)
    plt.ylim(min_y - .5 * max_y, max_y + .5 * max_y)
    if subplot == 4:
        plt.xlabel(u'第一主成分析')
        plt.ylabel(u'第二主成分析')
        plt.legend(loc="lower right")

In [145]:
plt.figure(figsize=(12, 7), facecolor='w')

X, Y = make_multilabel_classification(n_samples=100, n_features=30,
                                      n_classes=2, n_labels=2,
                                      allow_unlabeled=True,
                                      return_indicator=True,
                                      random_state=28)

plot_subfigure(X, Y, 1, u"包含不确定类别样本数据 + CCA", "cca")
plot_subfigure(X, Y, 2, u"包含不确定类别样本数据 + PCA", "pca")

X, Y = make_multilabel_classification(n_samples=100,  n_features=30,
                                      n_classes=2, n_labels=2,
                                      allow_unlabeled=False,
                                      return_indicator=True,
                                      random_state=28)

plot_subfigure(X, Y, 3, u"不包含不确定类别样本数据 + CCA", "cca")
plot_subfigure(X, Y, 4, u"不包含不确定类别样本数据 + PCA", "pca")

plt.subplots_adjust(.04, .02, .97, .94, .09, .2)
plt.show()

包含不确定类别样本数据 + CCA 准确率: 0.9
包含不确定类别样本数据 + PCA 准确率: 0.74
不包含不确定类别样本数据 + CCA 准确率: 0.98
不包含不确定类别样本数据 + PCA 准确率: 0.94
